In [1]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of records
n_samples = 1000

# Generation of artificial IoT data with boats
data = {
    "engine_temp": np.random.normal(loc=85, scale=10, size=n_samples),          # engine temperature, °C
    "battery_voltage": np.random.normal(loc=12.5, scale=0.5, size=n_samples),   # battery voltage, V
    "vibration_level": np.random.normal(loc=0.5, scale=0.2, size=n_samples),    # vibration level
    "fuel_level": np.random.uniform(10, 100, size=n_samples),                   # fuel level, %
    "rpm": np.random.normal(loc=3000, scale=500, size=n_samples),               # engine speed
    "gps_speed": np.random.normal(loc=6, scale=2, size=n_samples),              # speed, knots
    "error_code": np.random.choice(["None", "E01", "E02", "E03"], p=[0.9, 0.03, 0.04, 0.03], size=n_samples)
}

df = pd.DataFrame(data)

# Generate problem label (1 = potential fault, 0 = normal)
df["label"] = (
    (df["engine_temp"] > 95) |
    (df["battery_voltage"] < 12) |
    (df["vibration_level"] > 0.9) |
    (df["error_code"] != "None")
).astype(int)

df.to_csv('../data/telemetry.csv', index=False) 

df.head()

,engine_temp,battery_voltage,vibration_level,fuel_level,rpm,gps_speed,error_code,label
0,89.967142,13.199678,0.364964,83.686621,3090.531371,1.819394,None,0
1,83.617357,12.962317,0.471096,98.521049,3651.639019,5.659484,None,0
2,91.476885,12.529815,0.341516,33.370376,3293.082900,5.468791,None,0
3,100.230299,12.176532,0.438408,97.275127,2793.793091,5.200244,None,1
4,82.658466,12.849112,0.121277,48.811048,3128.599485,5.770718,None,0


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load data
df = pd.read_csv('../data/telemetry.csv')

features = [
    'engine_temp', 'battery_voltage', 'vibration_level',
    'fuel_level', 'rpm', 'gps_speed', 'error_code'
]

# Convert categorical error_code to numeric
df['error_code'] = df['error_code'].astype('category').cat.codes

X = df[features]
y = df['label']

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)

# Save model
joblib.dump(clf, '../model/model.pkl') 

['model.pkl']

Result
| N | engine_temp | battery_voltage | vibration_level | fuel_level |    rpm     | gps_speed | error_code | label |
|---|-------------|----------------|-----------------|------------|------------|-----------|------------|-------|
| 0 | 89.967142   | 13.199678      | 0.364964        | 83.686621  | 3090.531371| 1.819394  | None       | 0     |
| 1 | 83.617357   | 12.962317      | 0.471096        | 98.521049  | 3651.639019| 5.659484  | None       | 0     |
| 2 | 91.476885   | 12.529815      | 0.341516        | 33.370376  | 3293.082900| 5.468791  | None       | 0     |
| 3 | 100.230299  | 12.176532      | 0.438408        | 97.275127  | 2793.793091| 5.200244  | None       | 1     |
| 4 | 82.658466   | 12.849112      | 0.121277        | 48.811048  | 3128.599485| 5.770718  | None       | 0     |


In [17]:
import requests

url = "http://localhost:8000/predict"
payload = {
    "engine_temp": 100,
    "battery_voltage": 13.8,
    "vibration_level": 0.3,
    "fuel_level": 5,
    "rpm": 3200,
    "gps_speed": 4.1,
    "error_code": "None"
}

response = requests.post(url, json=payload)
print(response.json())

{'fault': True}
